Making plots for ICRC2023 Diversity Session

In [1]:
import altair as alt
import pandas as pd
import titanite as ti

print(f"Altair {alt.__version__}")
print(f"Pandas {pd.__version__}")
print(f"Titanite {ti.__version__}")

Altair 5.0.1
Pandas 2.0.3
Titanite 0.4.1


Read data

In [2]:
f_cfg = "../sandbox/config.toml"
f_csv = "../data/test_data/prepared_data.csv"
d = ti.Data(read_from=f_csv, load_from=f_cfg)
config = d.config()
data = d.read()
# data

2023-07-28 04:55:49.723 | INFO     | titanite.preprocess:categorical_data:126 - Categorize
2023-07-28 04:55:49.739 | INFO     | titanite.preprocess:binned_data:309 - Binned


`q13` の不要な値を削除する

In [3]:
q = "q13 >= 0"
q13_data = data.query(q)

Check correlation with `q13` and `q14`

In [4]:
def heatmap(data:pd.DataFrame, x:str, y:str):
    domain_x = alt.Scale(domain=[-10, 100])
    
    base = alt.Chart(data).encode(
        alt.Y(y),
    ).properties(
        width=400,
        height=400,
    )

    bin0 = base.mark_point().encode(
        alt.X(x).scale(domain_x)
    )

    bin5 = base.mark_rect().encode(
        alt.X(x).scale(domain_x).bin(step=5),
        alt.Color("count()").scale(scheme="blues"),
    )

    bin10 = base.mark_rect().encode(
        alt.X(x).scale(domain_x).bin(step=5),
        alt.Color("count()").scale(scheme="blues"),
    )

    hm = bin0 & bin5 & bin10
    return hm


In [5]:
heatmap(q13_data, x="q13", y="q14")

alt.VConcatChart(...)

In [6]:
q = "q14 != 'Prefer not to answer'"
q14_data = q13_data.query(q)
heatmap(q14_data, x="q13", y="q14")

alt.VConcatChart(...)

In [7]:
q = "q03_regional != 'Prefer not to answer'"
q03_data = q13_data.query(q)
heatmap(q03_data, x="q13", y="q03_regional")

alt.VConcatChart(...)

`q14`の値（割合）でクラスター化する

- ``10 - 20%``
- ``20 - 30%``
- ``30 - 40%``

In [25]:
def cluster_data(data:pd.DataFrame) -> pd.DataFrame:
    copied = data.copy()
    h = "q13_clustered"
    copied[h] = "Others"

    is_lower = copied["q13"] >= 0
    is_upper = copied["q13"] < 10
    isT = is_lower & is_upper
    copied.loc[isT, h] = "0% to 10%"

    is_lower = copied["q13"] >= 10
    is_upper = copied["q13"] < 20
    isT = is_lower & is_upper
    copied.loc[isT, h] = "10% to 20%"

    is_lower = copied["q13"] >= 20
    is_upper = copied["q13"] < 30
    isT = is_lower & is_upper
    copied.loc[isT, h] = "20% to 30%"

    is_lower = copied["q13"] >= 30
    is_upper = copied["q13"] < 40
    isT = is_lower & is_upper
    copied.loc[isT, h] = "30% to 40%"

    is_lower = copied["q13"] >= 40 
    is_upper = copied["q13"] < 110
    isT = is_lower & is_upper
    copied.loc[isT, h] = "40% and over"

    h = "q03_clustered"
    copied[h] = "Others"
    isT = copied["q03_regional"] == "Asia"
    copied.loc[isT, h] = "Asia"
    isT = copied["q03_regional"] == "Europe"
    copied.loc[isT, h] = "Europe"    
    isT = copied["q03_regional"] == "America"
    copied.loc[isT, h] = "America"

    h = "q04_clustered"
    copied[h] = "Others"
    isT = copied["q04_regional"] == "Asia"
    copied.loc[isT, h] = "Asia"
    isT = copied["q04_regional"] == "Europe"
    copied.loc[isT, h] = "Europe"    
    isT = copied["q04_regional"] == "America"
    copied.loc[isT, h] = "America"
    
    return copied

In [9]:
q13_data = cluster_data(q13_data)
q03_data = cluster_data(q03_data)
q04_data = cluster_data(q04_data)

In [10]:
q13_grouped, h13 = ti.core.hbar(q13_data, x="q14", color="q13_clustered", title="q13-q14")
q03_grouped, h03 = ti.core.hbar(q03_data, x="q13", color="q03_clustered", title="q03-q13")

In [11]:
h13 & h03

alt.VConcatChart(...)

クラスターごとの回答数で規格化する

In [12]:
q13_sum = q13_grouped.groupby("q13_clustered")["response"].sum().reset_index()
q13_merged = pd.merge(q13_grouped, q13_sum, on="q13_clustered")
q13_merged["response"] = q13_merged["response_x"] / q13_merged["response_y"]
#q13_merged

In [13]:
q03_sum = q03_grouped.groupby("q03_clustered")["response"].sum().reset_index()
q03_merged = pd.merge(q03_grouped, q03_sum, on="q03_clustered")
q03_merged["response"] = q03_merged["response_x"] / q03_merged["response_y"]
#q03_merged

In [14]:
g, h = ti.core.hbar(q13_merged, x="q14", color="q13_clustered", title="hoge")
h

alt.HConcatChart(...)

In [15]:
g, h = ti.core.hbar(q03_merged, x="q13", color="q03_clustered", title="hoge")
h

alt.HConcatChart(...)

In [16]:
heatmap(q03_data, x="q13", y="q03_clustered")

alt.VConcatChart(...)

# 発表用の図を（ちゃんと）作る

- ``X軸-Y軸``
- ``q14-q13_clustered``
- ``q13-q03_clustered``

In [17]:
# q13_merged

In [18]:
opacity = 0.7
#scheme = "category20b"
#scheme = "category20c"
scheme = "set1"
question = "【Q14】What do you think about the percentage of the female researcher in your group ?"
title_x = "What you think"
q = "q13_clustered != 'Others'"
q13_x = alt.Chart(q13_merged).mark_bar(opacity=opacity).encode(
    alt.X("q14").title(title_x),
    alt.Y("response_x").title("responses"),
    alt.Color("q13_clustered").title("Percentages").scale(scheme=scheme),
).properties(
    width=300,
    height=600,
)
#q13_x

In [19]:
q = "q13_clustered != 'Others'"
q13_n = alt.Chart(q13_merged.query(q)).mark_bar(opacity=opacity).encode(
    alt.X("q14").title("Answers"),
    alt.Y("response").title("normalized"),
    alt.Color("q13_clustered").title("Percentages").scale(scheme=scheme),
    alt.Row("q13_clustered").title("Females in your group"),
).properties(
    width=300,
    height=100,
)
q13_n

alt.Chart(...)

In [20]:
chart = (q13_x | q13_n).properties(title=question).configure_title(fontSize=20)
chart.save("../data/main_data/q14-q13_clustered.png")
chart

alt.HConcatChart(...)

``q13-q03_clustered``

In [21]:
# q03_merged

In [22]:
opacity = 0.7
scheme = "set1"
question = "【Q13】What is the percentage of female researcher in your group ?"
title_x = "Female researchers in your group [%]"

q03_x= alt.Chart(q03_merged).mark_bar(opacity=opacity).encode(    
    alt.X("q13").bin(step=10).title(title_x),
    alt.Y("response_x").title("entries"),
    alt.Color("q03_clustered").scale(scheme=scheme),
).properties(
    width=300,
    height=600,
)
#q03_x

In [23]:
q = "q03_clustered != 'Others'"
q03_n = alt.Chart(q03_merged).mark_bar(opacity=opacity).encode(
    alt.X("q13").bin(step=10).title(title_x),
    alt.Y("response").title("normalized"),
    alt.Color("q03_clustered:N").scale(scheme=scheme).title("Regions"),
    alt.Row("q03_clustered:N").title("Workplace Regions"),
).properties(
    width=300,
    height=100,
)
#q03_n

In [24]:
chart = (q03_x | q03_n).properties(title=question).configure_title(fontSize=20)
chart.save("../data/main_data/q13-q03_clustered.png")
chart

alt.HConcatChart(...)